## Avaliação Continuada (AC2) - Multi-Layer Perceptron

Nesse notebook, iremos ver como aplicar o método Multi-Layer Perceptron na nossa base de dados que foi processada no notebook `train_data.ipynb`.

Para começar então, vamos importar o notebook `train_data.ipynb` que irá conter tudo que precisamos.

In [1]:
import import_ipynb
import train_data

importing Jupyter notebook from train_data.ipynb


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415
Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

Algumas palavras que iremos usar para montar o nosso vocabulário:
['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
Tamanho do Vocabulario:  3632
[ 0.17063391 -0.12037469 -0.08291905  0.15739855  0.2682453  -0.33576694
  0.19767639  0.6196071  -0.18136293  0.23382549 -0.22981985 -0.37787122
  0.10857213  0.19396725 -0.09153517 -0.33911714 -0.073304   -0.0172643
  0.02148758 -0.48436213  0.22057416 -0.2971259  -0.1620915  -0.11847548
  0.08773892 -0.22823392 -0.04131408 -0.14172244 -0.32659385  0.14238487
  0.39600694  0.01346408  0.1605743  -0.08500466 -0.07830664  0.22776961
  0.27652663 -0.13253841 -0.06927069 -0.49206936 -0.32059664 -0.08952628
  0.00514935  0.11230633  0.49850705 -0.14280826 -0.21242063 -0.0868315
  0.155477    0.20335472  0.00258972 -0.1425103  -0.2185309  -0.2870042
  0.08256965 -0.09182009 -0.00823445 -0.30026847 -0.30620897  0.13255653
 -0.00526275  0.0479883

## Treinamento

Antes de começarmos o treinamento realmente, precisamos só mapear as nossas classes "spam" e "ham" para valores númericos para que a rede neural possa trabalhar com isso.

Se caso enviassemos um texto, "spam", ele iria mandar uma exceção durante a execução.

Além disso, também iremos preparar os valores de treino e teste e mapear eles para uma escala próximo de zero, com isso, melhoramos a precisão do nosso modelo.

Dessa forma, iremos criar duas funções para realizar essas transformações:

- `prepare_classes`: Prepara as classes (y).
- `prepare_values`: Prepara os valores (x)

In [2]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler

def prepare_classes(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)

	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)

	return y_train_enc, y_test_enc

def prepare_values(x_train, x_test):
	sc_X = StandardScaler()

	x_train_scaled = sc_X.fit_transform(x_train)
	x_test_scaled = sc_X.transform(x_test)

	return x_train_scaled, x_test_scaled

Com as funções definidas, vamos agora processar os nossos valores de teste e treino.

In [3]:
y_train_data, y_test_data = prepare_classes(train_data.y_train, train_data.y_test)
x_train_data, x_test_data = prepare_values(train_data.x_train, train_data.x_test)

Depois, podemos definir algumas variáveis para customizar o comportamento do nosso modelo.

- `hidden_layer_sizes`: Cada número representa o número de neuronios que será criado para a camada e cada item no vetor é uma camada. No exemplo abaixo, estaremos montando uma rede neural com 2 camadas, com 5 e 2 neuronios.
- `max_iter`: o número máximo de interações (também chamado de epochs)
- `tol`: o valor mínimo de melhoria após cada interação, se não melhorar ao menos isso, ele para a execução bem antes.
- `solver`: a estratégia que será usada para resolver o problema, abaixo, o valor será `lbfgs` (método quasi-newton). Segundo a [documentação do modelo](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#), para datasets menores, esse é um método recomendado e trás bons resultados.
- `activation`: a função de ativação, no caso, será a `relu`, que você pode ver mais informações [clicando aqui](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/).

In [4]:
hidden_layer_sizes = (5, 2)
max_iter = 2000
tol = 0.000001
solver = "lbfgs"
activation = "relu"

Depois, criamos o nosso modelo da seguinte forma:

In [13]:
from sklearn.neural_network import MLPClassifier

mlPerceptron = MLPClassifier(
  hidden_layer_sizes=hidden_layer_sizes,
  max_iter=max_iter,
  tol=tol,
  solver=solver,
  activation=activation,
  random_state=0
)

E ai, já podemos começar o treinamento do modelo.

In [14]:
mlPerceptron.fit(x_train_data, y_train_data) 

MLPClassifier(hidden_layer_sizes=(5, 2), max_iter=2000, random_state=0,
              solver='lbfgs', tol=1e-06)

E com o modelo treinado, podemos começar a ver as métricas de acurácia do nosso modelo tentando predizer os valores de teste que separamos na etapa anterior.

In [15]:
y_pred = mlPerceptron.predict(x_test_data)

Com os valores que foram classificados acima, podemos ver qual foi a sua performance usando o `sklearn` da seguinte forma:

In [16]:
from sklearn.metrics import classification_report

y_test_named = y_test_data.copy().astype(str)

y_test_named[y_test_named == "0"] = "Ham"
y_test_named[y_test_named == "1"] = "Spam"

y_pred_named = y_pred.copy().astype(str)
y_pred_named[y_pred_named == "0"] = "Ham"
y_pred_named[y_pred_named == "1"] = "Spam"

results = classification_report(y_test_named, y_pred_named)

print(results)

              precision    recall  f1-score   support

         Ham       0.98      0.98      0.98      1352
        Spam       0.84      0.85      0.85       196

    accuracy                           0.96      1548
   macro avg       0.91      0.91      0.91      1548
weighted avg       0.96      0.96      0.96      1548



Com a tabela acima, podemos tirar algumas conclusões como: 

E em comparação com os outros modelos, usando uma rede neural nós tivemos uma precisão maior na classe de Spam, assim como, uma acurácia muito boa também para o modelo como um todo.

Além disso, podemos checar a nossa acurácia com mais precisão tanto dos valores de teste quanto dos valores de treino da seguinte forma:

In [17]:
accuracyTest = train_data.accuracy_score(y_test_data, mlPerceptron.predict(x_test_data))
accuracyTrain = train_data.accuracy_score(y_train_data, mlPerceptron.predict(x_train_data))

print("Acurácia sobre os valores de teste: ", accuracyTest)
print("Acurácia sobre os valores de treino: ", accuracyTrain)

Acurácia sobre os valores de teste:  0.9612403100775194
Acurácia sobre os valores de treino:  1.0


E aqui vemos, temos, assim como mostrado no relatório anterior, certa de 96% e 100% de acurácia sobre os valores de teste e de treino, respectivamente.

## Cross-Validation

Apesar de eu não entender tão bem como funciona a validação cruzada, eu quis trazer nesses métodos para ter um pequena noção de como a ordem dos dados de treinamento influenciam o nosso modelo.

Dessa forma, o cross-validation permite que a gente separe em partições os nossos dados, e eles por consequencia, serão usados para treinar várias vezes o modelo e assim podemos obter as métrica de acurácia de cada vez que foi treinado o modelo.

Com essas métricas, podemos verificar se a mudança na ordem dos dados tem uma mudança significativa na acurácia do nosso modelo.

Vamos começar então com a importação das bibliotecas que iremos usar.

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

Depois, vamos criar algumas variáveis para configurarmos como será a cross-validation.

In [19]:
n_splits = 5 # o número de partições
test_size = 0.3 # a porcentagem que será usada para teste, o restante é usada no treino
cv = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=0) # define a função que irá realizar o rearranjo dos dados de treino e como será divido os dados.

E assim, podemos agora chamar a função `cross_val_score` que irá nos retornar a acurácia, assim como, o desvio da acurácia que nos diz quanto a ordem os dados está influenciando o modelo.

In [20]:
import numpy as np

x = np.concatenate([x_train_data, x_test_data], axis=0)
y = np.concatenate([y_train_data, y_test_data], axis=0)

scores = cross_val_score(mlPerceptron, x, y, cv=cv)

print("%0.2f acurácia com o desvio de %0.2f" % (scores.mean(), scores.std()))

0.96 acurácia com o desvio de 0.01


É interessante ver que agora a acurácia do modelo nem mudou de lugar, continuou nos 96% de antes.

Se comparado ao realizar isso com outros métodos, podemos perceber então que a ordem dos dados não influencia tanto a rede neural quanto afeta os outros métodos.